In [36]:
# Merging shapefiles with attribute preservation

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points

In [37]:
# Read multiple shapefiles and combine them into a single GeoDataFrame.
def read_and_combine_shapefiles(shapefiles):
    gdf_list = [gpd.read_file(file) for file in shapefiles]
    return gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# Transform CRS, create buffers, and merge by buffer
def process_geometries(gdf, tolerance):
    # Transform CRS to EPSG:31468
    gdf = gdf.to_crs(epsg=31468)
    # Create buffer around points
    gdf['buffer'] = gdf.geometry.buffer(tolerance)
    # Merge geometries by buffer
    merged_gdf = gdf.dissolve(by='buffer', aggfunc=lambda x: ', '.join(map(str, set(x))))
    # Remove temporary buffer column and transform CRS back to EPSG:4326
    return merged_gdf.reset_index().drop(columns=['buffer']).to_crs(epsg=4326)

# Filter GeoDataFrame to include only records with postal codes in Bavaria.
# def check_postal_codes_in_bavaria(gdf, bavaria_postcodes):
#     return gdf[gdf['postal_code'].isin(bavaria_postcodes)]

# Merge several shapefiles and save as a single shapefile."""
# def save_merged_shapefile(shapefiles, output_file, tolerance=10):
#     combined_gdf = read_and_combine_shapefiles(shapefiles)
#     processed_gdf = process_geometries(combined_gdf, tolerance)
#     # Check for postal codes in Bavaria and filter the GeoDataFrame
#     bavaria_postcodes = load_bavaria_postcodes()
#     filtered_gdf = check_postal_codes_in_bavaria(processed_gdf, bavaria_postcodes)
#     # Save the filtered GeoDataFrame to a shapefile
#     filtered_gdf.to_file(output_file, driver='ESRI Shapefile')

# Load Bavarian postal codes from a CSV file.
# def load_bavaria_postcodes():
#     df_postcodes = pd.read_csv('data/Germany_postcodes/Germany_postcodes.txt', sep='\t', header=None)
#     df_postcodes.columns = ['country_code', 'postal_code', 'place_name', 'admin_name1', 'admin_code1',
#                             'admin_name2', 'admin_code2', 'admin_name3', 'admin_code3',
#                             'latitude', 'longitude', 'accuracy']
#     # Filter for Bavaria (admin_name1 for Bavaria - "Bayern")
#     bavaria_postcodes = df_postcodes[df_postcodes['admin_name1'] == 'Bayern']
#     # Get the list of unique postal codes
#     return bavaria_postcodes['postal_code'].unique().tolist()

In [38]:
# A list of shapefiles for merging
shapefiles = [
    'data/sources/bundesnetzagentur_power_plants.shp',
    'data/sources/energieatlas_biomass.shp',
    'data/sources/energieatlas_biomethane.shp',
    'data/sources/energieatlas_biomethane.shp',
    'data/sources/energieatlas_geothermal.shp',
    'data/sources/energieatlas_kwk.shp',
    'data/sources/energieatlas_solarthermal.shp',
    'data/sources/energieatlas_waste.shp',
    'data/sources/OPSD_conventional_power_plants.shp',
    'data/sources/results_osm_search.shp'
]

In [39]:
# Reading and merging shapiefiles in the list in one GeoDataFrame
gdf_list = [gpd.read_file(file) for file in shapefiles]
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

In [40]:
# changing crs from epsg=4326 to epsg=31468 (DHDN / 3-degree Gauss-Kruger zone 4), which is used for Germany
# in epsg=31468 the tolerance is measured in meters, we get a circle buffer and no errors
# in epsg=4326 the tolerance is measured in degrees, we get an ellipse and an error
combined_gdf = combined_gdf.to_crs(epsg=31468)
# Create a temporary buffer around points to determine the points nearby
tolerance = 10
combined_gdf['buffer'] = combined_gdf.geometry.buffer(tolerance)

In [41]:
# Grouping by buffer. If the buffers of some points intersect,
# the points will be combined in one group
# 'dissolve' method aggregates attributes keeping all of them
aggregated_gdf = combined_gdf.dissolve(by='buffer', aggfunc=lambda x: ', '.join(map(str, set(x))))

# Delete temporary column 'buffer'
aggregated_gdf = aggregated_gdf.reset_index().drop(columns=['buffer'])

#Change crs back to epsg=4326
aggregated_gdf.to_crs(epsg=4326, inplace=True)

In [42]:
# aggregated_gdf = aggregated_gdf[aggregated_gdf['postcode'].isin(bavaria_postcodes)]

In [43]:
aggregated_gdf.head()

,geometry,MaStR-Nr.,Anlagenbet,Anzeige-Na,Postcode,City,Street,House Numb,State,Datum der,...,merge_comm,comment,generator_,plant_outp,generato_1,generato_2,generato_3,plant_ou_1,plant_ou_2,plant_ou_3
0,POINT (9.61198 47.58005),nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,POINT (9.60514 47.58062),nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,POINT (9.61370 47.57187),nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,POINT (9.64191 47.58671),nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,POINT (9.65257 47.59067),nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [44]:
# Name an output file as 'combined_sources.shp'
output_file = 'data/combined_sources/combined_sources.shp'
aggregated_gdf.to_file(output_file, driver='ESRI Shapefile')

Postcodes check

bayern_postcodes_overpass_turbo.geojson

In [45]:
combined_sources = gpd.read_file('data/combined_sources/combined_sources.shp')
bayern_postcodes_overpass_turbo = gpd.read_file('data/bayern_postcodes_overpass_turbo.geojson')

In [46]:
bayern_postcodes_overpass_turbo.head()

,id,@id,TMC:cid_58:tabcd_1:Class,boundary,fixme,name,name:sk,note,postal_code,postal_code_level,ref,service,source,type,wikidata,geometry
0,relation/387540,relation/387540,None,postal_code,None,None,None,63869 Heigenbrücken,63869,8,None,None,None,boundary,None,"POLYGON ((9.39894 50.01473, 9.39839 50.01475, ..."
1,relation/534826,relation/534826,None,postal_code,None,None,None,63856 Bessenbach,63856,8,None,None,None,boundary,None,"POLYGON ((9.24700 49.92629, 9.24736 49.92670, ..."
2,relation/534828,relation/534828,None,postal_code,None,None,None,63874 Dammbach,63874,8,None,None,None,boundary,None,"POLYGON ((9.28832 49.86678, 9.28812 49.86670, ..."
3,relation/534829,relation/534829,None,postal_code,None,None,None,63826 Geiselbach,63826,8,None,None,None,boundary,None,"POLYGON ((9.17480 50.11408, 9.17463 50.11411, ..."
4,relation/534847,relation/534847,None,postal_code,None,None,None,63864 Glattbach,63864,8,None,None,None,boundary,None,"POLYGON ((9.12653 50.01416, 9.12693 50.01370, ..."


In [47]:
aggregated_gdf_with_postcodes = gpd.sjoin(combined_sources, bayern_postcodes_overpass_turbo, how='left')
aggregated_gdf_with_postcodes.head()

,MaStR-Nr.,Anlagenbet,Anzeige-Na,Postcode,City,Street,House Numb,State,Datum der,Jahr der I,...,name,name:sk,note,postal_code,postal_code_level,ref,service,source,type_right,wikidata
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,None,None,88149 Nonnenhorn,88149,8,None,None,http://wiki.openstreetmap.org/wiki/Import/Cata...,boundary,None
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,None,None,88149 Nonnenhorn,88149,8,None,None,http://wiki.openstreetmap.org/wiki/Import/Cata...,boundary,None
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,None,None,88149 Nonnenhorn,88149,8,None,None,http://wiki.openstreetmap.org/wiki/Import/Cata...,boundary,None
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,None,None,88142 Wasserburg (Bodensee),88142,8,None,None,http://wiki.openstreetmap.org/wiki/Import/Cata...,boundary,None
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,None,None,88131 Lindau (Bodensee),88131,8,None,None,http://wiki.openstreetmap.org/wiki/Import/Cata...,boundary,None


In [48]:
test_qgis_1 = aggregated_gdf_with_postcodes[aggregated_gdf_with_postcodes['postal_code'].isna()]
test_qgis_1

,MaStR-Nr.,Anlagenbet,Anzeige-Na,Postcode,City,Street,House Numb,State,Datum der,Jahr der I,...,name,name:sk,note,postal_code,postal_code_level,ref,service,source,type_right,wikidata
151,"SEE907294548426, SEE922767690112, SEE918933000...","EEG-Anlagen < 10 MW, VERBUND Energy4Business G...","Scheuring K(Ro)1, None, Batterie Iphofen, Ober...","None, 86937, 97346, 86698, 86679, 86830","None, Schwabmünchen, Ellgau, Iphofen, Oberndor...",None,None,Bayern,"None, 1952-05-23, 1980-01-01, 1952-04-18, 2022...","1952.0, 1954.0, 2022.0, nan, nan, nan, nan, na...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3612,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9445,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9446,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11947,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
test_qgis_1.to_file('data/test_qgis_1.shp')

C:\Users\Natalia\AppData\Local\Temp\ipykernel_4604\769119782.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  test_qgis_1.to_file('data/test_qgis_1.shp')


In [51]:
# Delete points
aggregated_gdf_with_postcodes.drop([9445], inplace=True)
aggregated_gdf_with_postcodes.drop([151], inplace=True)

In [52]:
# a new location for one point
geom_1 = Point(12.218997, 48.582497)
# replace geometry for this point
aggregated_gdf_with_postcodes.loc[[9446], 'geometry'] = geom_1
#aggregated_gdf_with_postcodes.loc[9446]

In [73]:
# a new location for another point
geom_2 = Point(13.004090, 48.245847)
# replace geometry with new one
aggregated_gdf_with_postcodes.loc[[3612], 'geometry'] = geom_2
#aggregated_gdf_with_postcodes.loc[3612]
#aggregated_gdf_with_postcodes
#aggregated_gdf_with_postcodes.info()
#aggregated_gdf_with_postcodes.columns
aggregated_gdf_with_postcodes.dtypes
#aggregated_gdf_with_postcodes.isna().sum()

MaStR-Nr.     object
Anlagenbet    object
Anzeige-Na    object
Postcode      object
City          object
               ...  
ref           object
service       object
source        object
type_right    object
wikidata      object
Length: 130, dtype: object

In [54]:
output_file_corrected = 'data/combined_sources/combined_sources_corrected.shp'
aggregated_gdf_with_postcodes.to_file(output_file_corrected, driver='ESRI Shapefile')

C:\Users\Natalia\AppData\Local\Temp\ipykernel_4604\3355801671.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  aggregated_gdf_with_postcodes.to_file(output_file_corrected, driver='ESRI Shapefile')


bayern_postcodes_plz_5stellig

In [29]:
# bayern_postcodes_plz_5stellig = gpd.read_file('data/bayern_postcodes_plz_5stellig.geojson')

In [30]:
# aggregated_gdf_with_postcodes_2 = gpd.sjoin(combined_sources, bayern_postcodes_plz_5stellig, how='left')
# aggregated_gdf_with_postcodes_2

In [31]:
# test_qgis_2 = aggregated_gdf_with_postcodes_2[aggregated_gdf_with_postcodes_2['plz'].isna()]
# test_qgis_2

In [32]:
# test_qgis_2.to_file('data/test_qgis_2.shp')